<a href="https://colab.research.google.com/github/Kaleab-A/SignLanguage2LetterTranslater/blob/main/Sign_Language_2_Letter_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d grassknoted/asl-alphabet

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 1.02G/1.03G [00:06<00:00, 126MB/s]
100% 1.03G/1.03G [00:06<00:00, 173MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/asl-alphabet.zip"

In [ ]:
import keras
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils.np_utils import to_categorical
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import os
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import skimage
import csv
from tqdm import tqdm
from sklearn import model_selection
%matplotlib inline

In [ ]:
imageSize=50
train_dir = "/content/asl_alphabet_train/asl_alphabet_train/"
test_dir =  "/content/asl_alphabet_test/asl_alphabet_test/"
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['A']:
                label = 0
            elif folderName in ['B']:
                label = 1
            elif folderName in ['C']:
                label = 2
            elif folderName in ['D']:
                label = 3
            elif folderName in ['E']:
                label = 4
            elif folderName in ['F']:
                label = 5
            elif folderName in ['G']:
                label = 6
            elif folderName in ['H']:
                label = 7
            elif folderName in ['I']:
                label = 8
            elif folderName in ['J']:
                label = 9
            elif folderName in ['K']:
                label = 10
            elif folderName in ['L']:
                label = 11
            elif folderName in ['M']:
                label = 12
            elif folderName in ['N']:
                label = 13
            elif folderName in ['O']:
                label = 14
            elif folderName in ['P']:
                label = 15
            elif folderName in ['Q']:
                label = 16
            elif folderName in ['R']:
                label = 17
            elif folderName in ['S']:
                label = 18
            elif folderName in ['T']:
                label = 19
            elif folderName in ['U']:
                label = 20
            elif folderName in ['V']:
                label = 21
            elif folderName in ['W']:
                label = 22
            elif folderName in ['X']:
                label = 23
            elif folderName in ['Y']:
                label = 24
            elif folderName in ['Z']:
                label = 25
            elif folderName in ['del']:
                label = 26
            elif folderName in ['nothing']:
                label = 27
            elif folderName in ['space']:
                label = 28           
            else:
                label = 29
            for image_filename in tqdm(os.listdir(folder + folderName)):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y
X_train, y_train = get_data(train_dir) 
#X_test, y_test= get_data(test_dir) # Too few images

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2) 

# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
y_trainHot = to_categorical(y_train, num_classes = 30)
y_testHot = to_categorical(y_test, num_classes = 30)

In [ ]:
!cp "/content/drive/MyDrive/Google Colab Files/X_test.npy" "/content/"
!cp "/content/drive/MyDrive/Google Colab Files/X_train.npy" "/content/"
!cp "/content/drive/MyDrive/Google Colab Files/y_trainHot.npy" "/content/"
!cp "/content/drive/MyDrive/Google Colab Files/y_testHot.npy" "/content/"

In [ ]:
X_train = np.load("X_train.npy")

In [ ]:
X_test = np.load("X_test.npy")
y_trainHot = np.load("y_trainHot.npy")
y_testHot = np.load("y_testHot.npy")


In [ ]:
def model_vgg3():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_initializer="he_uniform", input_shape=(50, 50, 3)))
    model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', kernel_initializer="he_uniform"))
    model.add(MaxPooling2D(pool_size=2))
    # model.add(Dropout(0.2))

    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer="he_uniform"))
    model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer="he_uniform"))
    model.add(MaxPooling2D(pool_size=2))
    # model.add(Dropout(0.3))

    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', kernel_initializer="he_uniform"))
    model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', kernel_initializer="he_uniform"))
    model.add(MaxPooling2D(pool_size=2))
    # model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(30, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
    return model    


In [ ]:
filePath = "vgg3_adam_sparse.hdf5"

In [ ]:
print(X_train.shape, y_trainHot.shape)

(69600, 50, 50, 3) (69600, 30)


In [ ]:
model = model_vgg3()
stop_early = EarlyStopping(monitor="val_loss", patience=5)

checkpointer = ModelCheckpoint(filepath=filePath, verbose=1, 
                            save_best_only=True)
hist = model.fit(X_train, y_trainHot, batch_size=64, epochs=20, validation_split=0.2, callbacks=[checkpointer, stop_early], shuffle=True)

Epoch 1/20
869/870 [============================>.] - ETA: 0s - loss: 1.1505 - accuracy: 0.6509
Epoch 1: val_loss improved from inf to 0.23805, saving model to vgg3_adam_sparse.hdf5
870/870 [==============================] - 59s 55ms/step - loss: 1.1495 - accuracy: 0.6513 - val_loss: 0.2380 - val_accuracy: 0.9223
Epoch 2/20
870/870 [==============================] - ETA: 0s - loss: 0.0983 - accuracy: 0.9678
Epoch 2: val_loss improved from 0.23805 to 0.04930, saving model to vgg3_adam_sparse.hdf5
870/870 [==============================] - 45s 52ms/step - loss: 0.0983 - accuracy: 0.9678 - val_loss: 0.0493 - val_accuracy: 0.9828
Epoch 3/20
870/870 [==============================] - ETA: 0s - loss: 0.0482 - accuracy: 0.9850
Epoch 3: val_loss improved from 0.04930 to 0.02533, saving model to vgg3_adam_sparse.hdf5
870/870 [==============================] - 48s 55ms/step - loss: 0.0482 - accuracy: 0.9850 - val_loss: 0.0253 - val_accuracy: 0.9920
Epoch 4/20
869/870 [===========================

In [ ]:
model.evaluate(X_test, y_testHot)

544/544 [==============================] - 7s 13ms/step - loss: 0.0354 - accuracy: 0.9874


[0.03535397723317146, 0.9873563051223755]